In [1]:
import numpy as np
import sys
import itertools as itertools
sys.path.append("..")
sys.path.append("../..")

import matplotlib.pyplot as plt
from python.Hubbard import Hubbard_model_with_filling, Double_occupancy, get_filling
from python.DMRG import DMRG
from python.Zippers import MPO_to_Hamiltonian, contract_MPS
from python.Canonical_Form import get_Neumann_entropy
from python.Backend import Backend
from python.Decomposition import EIGH

In [13]:
n_sites = 10

hopping_t = 1.0
interaction_U = 2
chemical_potential = 1
# bk = Backend("numpy")
bk = Backend("torch")

Hamiltonian = Hubbard_model_with_filling(
    bk=bk, n_sites=n_sites, hopping_t=hopping_t,
    interaction_U=interaction_U, chemical_potential=chemical_potential,
)

MPO_transposed = [bk.transpose(ham, (2, 3, 0, 1)) for ham in Hamiltonian]

In [14]:
# """
# Get exact matrix (exponential barrier)
# """

# matrix = MPO_to_Hamiltonian(MPO_transposed, bk=bk)
# eigvals, _ = EIGH(matrix, bk=bk)
# eigvals

In [15]:
# np.linalg.norm(matrix - matrix.conj().T)

In [16]:
NKeep = 20
NSweep = 10
Krylov_bases = 5 #* 5 is usually enough
Lanczos_cutoff = 1e-2
two_site = True
verbose = True

In [17]:
ground_energies, ground_times, ground_MPS = DMRG(
    bk = bk,
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = True,
    two_site = two_site,
    verbose = verbose,
)

# eigvals[0]

L=10 | NKeep=20 | NSweep=10 | diag=True | two=True | Krylov=5 | cutoff=0.01
Iterative diagonalization complete
iter=0 | energy=-17.726795 | time=0js
iter=1 | energy=(-17.906319398714942+0j) | time=1.1656220750001012s
iter=2 | energy=(-17.90632159151029+0j) | time=2.3055384680001225s
iter=3 | energy=(-17.90632160527373+0j) | time=3.4700943859998006s


In [18]:
excited_energies, excited_times, excited_MPS = DMRG(
    bk = bk,
    Hamiltonian = Hamiltonian,
    NKeep = NKeep,
    NSweep = NSweep,
    Krylov_bases = Krylov_bases,
    Lanczos_cutoff = Lanczos_cutoff,
    iterative_diag = False,
    orthogonal_to_list_of_MPS = [ground_MPS],
    two_site = two_site,
    verbose = verbose,
)

# eigvals[1]

L=10 | NKeep=20 | NSweep=10 | diag=False | two=True | Krylov=5 | cutoff=0.01
Random initialization complete
iter=0 | energy=-5.0347099 | time=0js
iter=1 | energy=(-17.516613549234755+0j) | time=1.7298197570007687s
iter=2 | energy=(-17.525923158119756+0j) | time=3.3220735770009924s
iter=3 | energy=(-17.527650289662095+0j) | time=4.942420479001157s
iter=4 | energy=(-17.527918502611264+0j) | time=6.535564974001318s
iter=5 | energy=(-17.5280790845565+0j) | time=8.137189887002023s
iter=6 | energy=(-17.528312962742152+0j) | time=9.751689259001978s
iter=7 | energy=(-17.52860867400795+0j) | time=11.356875524002135s
iter=8 | energy=(-17.5286206232876+0j) | time=13.01602300900231s
iter=9 | energy=(-17.528625541971934+0j) | time=14.617983292002464s
iter=10 | energy=(-17.528627886880304+0j) | time=16.06172419500308s


In [19]:
get_Neumann_entropy(ground_MPS, bk = bk,)

tensor([1.8550+0.j, 1.2903+0.j, 1.9581+0.j, 1.5058+0.j, 1.9874+0.j, 1.5060+0.j, 1.9607+0.j,
        1.2917+0.j, 1.8551+0.j], device='cuda:0')

In [20]:
get_Neumann_entropy(excited_MPS, bk = bk,)

tensor([1.5785+0.j, 1.5160+0.j, 1.4522+0.j, 1.9189+0.j, 1.4030+0.j, 1.9193+0.j, 1.4524+0.j,
        1.5165+0.j, 1.5789+0.j], device='cuda:0')

In [21]:
get_filling(ground_MPS, bk = bk,).sum()

tensor(10.0000+0.j, device='cuda:0')

In [22]:
get_filling(excited_MPS, bk = bk,).sum()

tensor(10.0000-2.4376e-16j, device='cuda:0')